<a href="https://colab.research.google.com/github/farrelrassya/nlpspacybeginners/blob/main/3.%20WordVectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

# To filter warnings based on their category
# For example, to ignore all DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# To filter warnings based on their message
# For example, to ignore a specific warning message
warnings.filterwarnings("ignore", message="Specific warning message")

# Your code goes here

# Resetting the warning filters after the code block
warnings.resetwarnings()

In this notebook is word vectors, or word embeddings. Because the English small model does not have these saved, we will be working with the next largest model, the English medium model, en_core_web_md. Let’s import spaCy and download the medium model.

In [4]:
import spacy
!python -m -q spacy download en_core_web_md

/usr/bin/python3: No module named -q


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
warnings.filterwarnings("ignore", category=ImportWarning)
nlp = spacy.load("en_core_web_md")
with open ("wiki_us.txt", "r") as f:
    text = f.read()
doc = nlp(text)
sentence1 = list(doc.sents)[0]

##Word Vectors

Word vectors, or word embeddings, are numerical representations of words in multidimensional space through matrices. The purpose of the word vector is to get a computer system to understand a word. Computers cannot understand text efficiently. They can, however, process numbers quickly and well. For this reason, it is important to convert a word into a number.

Initial methods for creating word vectors in a pipeline take all words in a corpus and convert them into a single, unique number. These words are then stored in a dictionary that would look like this: {“the”: 1, “a”, 2} etc. This is known as a bag of words. This approach to representing words numerically, however, only allow a computer to understand words numerically to identify unique words. It does not, however, allow a computer to understand meaning.

Imagine this scenario:

1. Tom loves to eat chocolate.

2. Tom likes to eat chocolate.

These sentences represented as a numerical array (list) would look like this:

1. 1, 2, 3, 4, 5

2. 1, 6, 3, 4, 5

As we can see, as humans both sentences are nearly identical. The only difference is the degree to which Tom appreciates eating chocolate. If we examine the numbers, however, these two sentences seem quite close, but their semantical meaning is impossible to know for certain. How similar is 2 to 6? The number 6 could represent “hates” as much as it could represent “likes”. This is where word vectors come in.

Word vectors take these one dimensional bag of words and gives them multidimensional meaning by representing them in higher dimensional space, noted above. This is achieved through machine learning and can be easily achieved via Python libraries, such as Gensim, which we will explore more closely in the next notebook.



The goal of word vectors is to achieve numerical understanding of language so that a computer can perform more complex tasks on that corpus. Let’s consider the example above. How do we get a computer to understand 2 and 6 are synonyms or mean something similar? One option you might be thinking is to simply give the computer a synonym dictionary. It can look up synonyms and then know what words mean. This approach, on the surface, makes perfect sense, but let’s explore that option and see why it cannot possibly work.

For the example below, we will be using the Python library PyDictionary which allows us to look up definitions and synonyms of words.

In [7]:
"""
from PyDictionary import PyDictionary

dictionary=PyDictionary()
text = "Tom loves to eat chocolate"

words = text.split()
for word in words:
    syns = dictionary.synonym(word)
    print (f"{word}: {syns[0:5]}\n")

"""
"""
possible output will be:
Tom has no Synonyms in the API
warning:
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-17-2fe432a3b47a> in <module>
      7 for word in words:
      8     syns = dictionary.synonym(word)
----> 9     print (f"{word}: {syns[0:5]}\n")

TypeError: 'NoneType' object is not subscriptable
"""

'\npossible output will be:\nTom has no Synonyms in the API\nwarning:\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-17-2fe432a3b47a> in <module>\n      7 for word in words:\n      8     syns = dictionary.synonym(word)\n----> 9     print (f"{word}: {syns[0:5]}\n")\n\nTypeError: \'NoneType\' object is not subscriptable\n'

Even with the simple sentence, the results are comically bad. Why? The reason is because synonym substitution, a common method of data augmentation, does not take into account syntactical differences of synonyms. I do not believe anyone would think “Felis domesticus”, the Latin name of the common house cat, would be an adequite substitution for the name Tom. Nor is “garbage down” a really proper synonym for eat.

Perhaps, then we could use synonyms to find words that have cross-terms, or terms that appear in both synonym sets.

In [8]:
"""
from PyDictionary import PyDictionary

dictionary=PyDictionary()

words  = ["like", "love"]
for word in words:
    syns = dictionary.synonym(word)
    print (f"{word}: {syns[0:5]}\n")

output : like has no Synonyms in the API
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-18-fefd3418da3d> in <module>
      6 for word in words:
      7     syns = dictionary.synonym(word)
----> 8     print (f"{word}: {syns[0:5]}\n")

TypeError: 'NoneType' object is not subscriptable
"""

'\nfrom PyDictionary import PyDictionary\n\ndictionary=PyDictionary()\n\nwords  = ["like", "love"]\nfor word in words:\n    syns = dictionary.synonym(word)\n    print (f"{word}: {syns[0:5]}\n")\n\noutput : like has no Synonyms in the API\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-18-fefd3418da3d> in <module>\n      6 for word in words:\n      7     syns = dictionary.synonym(word)\n----> 8     print (f"{word}: {syns[0:5]}\n")\n\nTypeError: \'NoneType\' object is not subscriptable\n'

This, as we can see, has some potential to work, but again it is not entirely reliable and to work with such a list would be computationally expensive. For both of these reasons, word vectors are prefered. The reason? Because they are formed by the computer on corpora for a specific task. Further, they are numerical in nature (not a dictionary of words), meaning the computer can process them more quickly.

Word vectors have a preset number of dimensions. These dimensions are honed via machine learned. Models take into account word frequency alongside words across a corpus and the appearance of other words in similar contexts. This allows for the the computer to determine the syntactical similarity of words numerically. It then needs to represent these relationships numerically. It does this through the vector, or a matrix of matrices. To represent these more concisely, models flatten a matrix to a float (decimal number). The number of dimensions represent the number of floats in the matrix.

Let’s take a look at the first word in our sentence. Specifically, let’s look at its vector.

In [9]:
sentence1[0].vector

array([-7.2681e+00, -8.5717e-01,  5.8105e+00,  1.9771e+00,  8.8147e+00,
       -5.8579e+00,  3.7143e+00,  3.5850e+00,  4.7987e+00, -4.4251e+00,
        1.7461e+00, -3.7296e+00, -5.1407e+00, -1.0792e+00, -2.5555e+00,
        3.0755e+00,  5.0141e+00,  5.8525e+00,  7.3378e+00, -2.7689e+00,
       -5.1641e+00, -1.9879e+00,  2.9782e+00,  2.1024e+00,  4.4306e+00,
        8.4355e-01, -6.8742e+00, -4.2949e+00, -1.7294e-01,  3.6074e+00,
        8.4379e-01,  3.3419e-01, -4.8147e+00,  3.5683e-02, -1.3721e+01,
       -4.6528e+00, -1.4021e+00,  4.8342e-01,  1.2549e+00, -4.0644e+00,
        3.3278e+00, -2.1590e-01, -5.1786e+00,  3.5360e+00, -3.1575e+00,
       -3.5273e+00, -3.6753e+00,  1.5863e+00, -8.1594e+00, -3.4657e+00,
        1.5262e+00,  4.8135e+00, -3.8428e+00, -3.9082e+00,  6.7549e-01,
       -3.5787e-01, -1.7806e+00,  3.5284e+00, -5.1114e-02, -9.7150e-01,
       -9.0553e-01, -1.5570e+00,  1.2038e+00,  4.7708e+00,  9.8561e-01,
       -2.3186e+00, -7.4899e+00, -9.5389e+00,  8.5572e+00,  2.74

Why use Word Vectors?
Once a word vector model is trained, we can do similarity matches very quickly and very reliably. Let’s explore some vectors from our medium sized model. Let’s specifically try and find the words most closely related to the word dog.


In [10]:
#https://stackoverflow.com/questions/54717449/mapping-word-vector-to-the-most-similar-closest-word-using-spacy
import numpy as np
your_word = "dog"

ms = nlp.vocab.vectors.most_similar(
    np.asarray([nlp.vocab.vectors[nlp.vocab.strings[your_word]]]), n=10)
words = [nlp.vocab.strings[w] for w in ms[0][0]]
distances = ms[2]
print(words)

['dogsbody', 'wolfdogs', 'Baeg', 'duppy', 'pet(s', 'postcanine', 'Kebira', 'uppies', 'Toropets', 'moggie']


In spaCy we can do this same thing at the document level. Through word vectors we can calculate the similarity between two documents. Let’s look at the example from spaCy’s documentation.

In [11]:
nlp = spacy.load("en_core_web_md")  # make sure to use larger package!
doc1 = nlp("I like salty fries and hamburgers.")
doc2 = nlp("Fast food tastes very good.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))

I like salty fries and hamburgers. <-> Fast food tastes very good. 0.691649353055761


We can also calculate the word similarity between two given words.

In [12]:
# Similarity of tokens and spans
french_fries = doc1[2:4]
burgers = doc1[5]
print(french_fries, "<->", burgers, french_fries.similarity(burgers))

salty fries <-> hamburgers 0.6938489675521851
